# LangGraph Tutorial: Holiday Planner

### Necessary installations and the imports

In [6]:
!pip install langgraph langchain langchain-community langchain-openai python-dotenv --quiet

In [15]:
import os
os.environ["OPENAI_API_KEY"] = "Add your Open AI API key"

In [16]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END
import requests
from typing import List, Literal
from langchain_openai import ChatOpenAI

### Define the tools we are going to use

In [17]:
from langchain_core.tools import tool

# Define tools
@tool
def suggest_destination(preference: str) -> str:
    """
    Suggest a travel destination based on the user's preference.
    Supported preferences: 'beach', 'mountains', 'city'.
    """
    destinations = {
        "beach": ["Maldives", "Hawaii", "Bali"],
        "mountains": ["Swiss Alps", "Himalayas", "Rockies"],
        "city": ["New York", "Paris", "Tokyo"]
    }
    if preference.lower() in destinations:
        return f"Recommended destinations for {preference}: {', '.join(destinations[preference.lower()])}."
    return "Unknown Preference"

@tool
def calculate_cost(destination: str) -> str:
    """
    Calculate the estimated travel cost for a destination.
    Returns cost in USD.
    """
    costs = {
        "Maldives": 2000,
        "Hawaii": 2500,
        "Bali": 1800,
        "Swiss Alps": 3000,
        "Himalayas": 1500,
        "Rockies": 2200,
        "New York": 1500,
        "Paris": 2800,
        "Tokyo": 3200
    }
    if destination in costs:
        return f"The estimated cost for traveling to {destination} is ${costs[destination]}."
    return f"Cost data not available for {destination}."

@tool
def check_weather(destination: str) -> str:
    """
    Check the current weather for a destination.
    Assumes static data for demonstration.
    """
    weather = {
        "Maldives": "Sunny",
        "Hawaii": "Cloudy",
        "Bali": "Rainy",
        "Swiss Alps": "Snowy",
        "Himalayas": "Windy",
        "Rockies": "Sunny",
        "New York": "Rainy",
        "Paris": "Cloudy",
        "Tokyo": "Sunny"
    }
    if destination in weather:
        return f"The weather in {destination} is {weather[destination]}."
    return f"Weather data not available for {destination}."

In [18]:
# Add tools to a ToolNode
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

tools = [suggest_destination, calculate_cost, check_weather]
tool_node = ToolNode(tools)

# Define functions for LLM and tools
from langchain_openai import ChatOpenAI

def call_model(state: MessagesState):
    messages = state["messages"]
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # The API key will be read from the environment
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def call_tools(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

# Workflow setup
workflow = StateGraph(MessagesState)

workflow.add_node("LLM", call_model)
workflow.add_edge(START, "LLM")
workflow.add_node("tools", tool_node)
workflow.add_conditional_edges("LLM", call_tools)
workflow.add_edge("tools", "LLM")

agent = workflow.compile()

In [19]:
for chunk in agent.stream(
    {"messages": [("user", "My budget is 4000 and I love mountains. Suggest me a travel plan with atleast 2 destination ")]},
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

My budget is 4000 and I love mountains. Suggest me a travel plan with atleast 2 destination 
================================== Ai Message ==================================
Tool Calls:
  suggest_destination (call_Ru46XzC6Plf1xK9TQu8p74es)
 Call ID: call_Ru46XzC6Plf1xK9TQu8p74es
  Args:
    preference: mountains
================================= Tool Message =================================
Name: suggest_destination

Recommended destinations for mountains: Swiss Alps, Himalayas, Rockies.
================================== Ai Message ==================================
Tool Calls:
  calculate_cost (call_SwSEoh5OCw9ttuyRw0r5ZW8H)
 Call ID: call_SwSEoh5OCw9ttuyRw0r5ZW8H
  Args:
    destination: Swiss Alps
  calculate_cost (call_8d0GZawaJcW9SE1AXY583T4w)
 Call ID: call_8d0GZawaJcW9SE1AXY583T4w
  Args:
    destination: Himalayas
  calculate_cost (call_dCDl1Y1f1oNqdjILsujBwRsb)
 Call ID: call_dCDl1Y1f1oNqdjILsu